In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

Прочитаем данные и посмотрим что там вообще

In [27]:
data = pd.read_csv("./data/insurance_nans.csv")

In [33]:
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19.0,female,27.900,0.0,yes,southwest,16884.9240
1,18.0,male,33.770,1.0,no,NaN,1725.5523
2,28.0,male,33.000,3.0,no,southeast,NaN
3,33.0,male,22.705,0.0,no,northwest,NaN
4,32.0,male,28.880,0.0,no,northwest,3866.8552


pd.dropna, pd.fillna